# Mediation Analysis


A mediation analysis is conducted when a researcher is interested in the **mechanism** underlying how X has an effect on Y. It attempts to make a causal inference that an effect of some variable is mediated by the mediating variable.

Consider the instance below where X has an effect of $\beta_1$ on Y:

### 1) $Y = \beta_1 + c \cdot x $

In this model, there may be a third variable **M** which mediates the effect of X on Y. In other words, the variable M is partially responsible for the effect X has on Y. This is described here:

￼To conduct a mediation analysis one estimates two additional models: 1) the effect of X on M, and 2) the effect of X and M on Y. 

### 2) $M = \beta_2 + a \cdot X $

### 3) $y = \beta_3 + c' \cdot X + b \cdot M $  
  
  
Now the original effect of X on Y, denoted as **C**,can be broken down into two parts:

### $c = a \cdot b + c' $

$ a \cdot b $ is the indirect effect of X on Y via the mediator. 
$c'$ is the remaining direct effect of X on Y controlling for M.  

This relationship is depicted below.

<img src="Figures/mediation1.png",width=500,align='center'>

## Simulate a mediation

In this section we will simulate a mediation.

This is a case in which the true effect of X on Y is positive, but appears negative without testing for mediation. 

In [245]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import statsmodels.api as sm
from scipy import stats

def sobel_test(ea,eb,sea,seb):
    '''
    Sobel test for significance of mediation
    Input: 
    A: coefficient from X to mediator variable, M
    B: coefficient from M to Y 
    seA: Standard error of A
    seB: Standard error fo B
    Output: 
    t: Sobel's test statistic
    pval : Two-tailed probability assuming normal distribution
    '''
    SE = np.sqrt( (ea**2)*(sea**2) + (eb**2)*(seb**2))
    t = (ea*eb) / SE
    n = 100000000
    pval = stats.t.sf(np.abs(t), n-1)*2
    return t, pval

In [160]:
# set random seed so everyone gets same results
np.random.seed(1)
# Determine effects
a = -3 # effect of x to M
b = 3 # effect of M to y
cq = 8 # direct effect of x on y

# Create a random data x
x = np.random.rand(100) 
m = x * a + np.random.rand(100)

# Create Y
y = np.dot(np.array([x,m]).T,[cq,b]) + np.random.rand(100)

### 1) Test effect of X on Y

In [161]:
X = sm.add_constant(x)
lm1 = smf.OLS(y,X).fit()
print lm1.summary()
ec = lm1.params[1] # save total effect c to ec

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.097
Model:                            OLS   Adj. R-squared:                  0.088
Method:                 Least Squares   F-statistic:                     10.53
Date:                Sun, 15 May 2016   Prob (F-statistic):            0.00161
Time:                        16:22:18   Log-Likelihood:                -142.55
No. Observations:                 100   AIC:                             289.1
Df Residuals:                      98   BIC:                             294.3
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const          2.0678      0.196     10.539      0.0

### 2) Test effect of X on M

In [194]:
X = sm.add_constant(x)
lm2 = smf.OLS(m,X).fit()
print lm2.summary()
ea = lm2.params[1] # Save the effect of X on M, a, to ea
sea = lm2.bse[1]

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.888
Model:                            OLS   Adj. R-squared:                  0.887
Method:                 Least Squares   F-statistic:                     778.6
Date:                Sun, 15 May 2016   Prob (F-statistic):           2.01e-48
Time:                        16:51:40   Log-Likelihood:                -26.736
No. Observations:                 100   AIC:                             57.47
Df Residuals:                      98   BIC:                             62.68
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const          0.4978      0.062      8.078      0.0

### 3) Test effect of  X and M on Y 

In [197]:
X = np.array([x,m]).T
X = sm.add_constant(X) 
lm3 = smf.OLS(y,X).fit()
print lm3.summary()
ecq,eb = lm3.params[1:3]
seb = lm3.bse[2]

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.932
Model:                            OLS   Adj. R-squared:                  0.931
Method:                 Least Squares   F-statistic:                     666.5
Date:                Sun, 15 May 2016   Prob (F-statistic):           2.11e-57
Time:                        16:52:14   Log-Likelihood:                -13.113
No. Observations:                 100   AIC:                             32.23
Df Residuals:                      97   BIC:                             40.04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const          0.5435      0.070      7.790      0.0

### Show how the effect is broken down to direct and indirect effects
### Recall how the overall effect is decomposed
### $c = a \cdot b + c' $

In [164]:
print 'c : ', ec
print 'a : ', ea
print 'b : ', eb
print 'c\' : ', ecq

c :  -1.12055253349
a :  -3.02671026091
b :  3.0620641022
c' :  8.14742830421


In [168]:
print 'Total effect C:', ec
print 'is decomposed into the indirect(mediated) effect a*b :', ea*eb
print 'plus the direct effect c\'', ecq
print 'which adds up to ', ea*eb+ecq

Total effect C: -1.12055253349
is decomposed into the indirect(mediated) effect a*b : -9.2679808377
plus the direct effect c' 8.14742830421
which adds up to  -1.12055253349


# Run a Sobel Test for Significance of Mediation

In [248]:
t,p = sobel_test(ea,eb,sea,seb)
print 'Sobel\'s test of significance t = %2.2f' %t
print 'Two-tailed p-value p = %2.5f ' %p

Sobel's test of significance t = -21.76
Two-tailed p-value p = 0.00000 


# Moderation Analysis

In a moderation analysis, the moderator is a variable that explains the relationship between two other variables, similar to an interaction term. Moderation is slightly different from an interaction due to the additional constraint that there is a causal relationship from X to Y BUT not from Z to Y. Therefore, a moderation implies an interaction exists but an interaction does not imply a moderation. 

To recap, a mediating variable we just covered is one that influences the strength of X on Y. 

Here is a schematic representation of a moderation relationship. 
This diagram hypothesize that Stress has a causal relationship to Depression 
but the effect of Stress is different for people with high or low Social Support 

<img src="Figures/moderator2.gif",width=500,align='center'>

This can be reprsented by an interaction, 



<img src="Figures/moderator3.jpeg",width=500,align='center'>


Examples have been retrieved from [here](http://www.victoria.ac.nz/psyc/paul-jose-files/helpcentre/help5_moderation_example.php)

In [292]:
df = {'enjoy':[4,15,1,11,13,19,6,10,15,3,11,20,7,11,10,6,7,18,8,2,7,12,13,15,4,3,9,7,10,2,15,1,3,6,13,15,16,14,1,8],
'buy':[16,19,0,19,25,24,22,21,13,7,28,31,4,26,11,12,14,16,20,13,12,23,22,19,12,10,7,22,7,0,16,17,11,5,29,29,20,16,3,8
],
'read':[6,13,1,13,12,11,7,8,12,4,15,14,7,14,9,5,7,12,10,6,9,13,9,13,9,5,7,8,8,2,7,6,9,9,15,11,9,7,2,10
]}
df = pd.DataFrame(df)
df['int'] = df.read*df.buy
df.corr()
df['normint'] = ((df.read-np.mean(df.read))/np.std(df.read))*((df.buy-np.mean(df.buy))/np.std(df.buy))

In [293]:
np.corrcoef(df.buy,df.int)

array([[ 1.        ,  0.91944697],
       [ 0.91944697,  1.        ]])

In [294]:
np.corrcoef(df.buy,df.normint)

array([[ 1.        , -0.07935977],
       [-0.07935977,  1.        ]])

In [299]:
mod = smf.ols(formula = "enjoy ~ buy + read + normint", data = df).fit()
print mod.summary()

                            OLS Regression Results                            
Dep. Variable:                  enjoy   R-squared:                       0.559
Model:                            OLS   Adj. R-squared:                  0.523
Method:                 Least Squares   F-statistic:                     15.23
Date:                Sun, 15 May 2016   Prob (F-statistic):           1.47e-06
Time:                        18:28:35   Log-Likelihood:                -106.98
No. Observations:                  40   AIC:                             222.0
Df Residuals:                      36   BIC:                             228.7
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept     -0.3201      1.682     -0.190      0.8

Another example with a significant moderator

In [334]:
df = pd.read_csv('../Data/salary.csv')
df = df.loc[df.gender==0]
df['dept_num'] = df.departm.map({'bio':0, 'chem':1,'geol':2,'neuro':3,'stat':4,'physics':5,'math':6})
lm = smf.ols(formula = "salary ~ years + departm + departm * years",data=df).fit()
print lm.summary()

                            OLS Regression Results                            
Dep. Variable:                 salary   R-squared:                       0.360
Model:                            OLS   Adj. R-squared:                  0.200
Method:                 Least Squares   F-statistic:                     2.248
Date:                Sun, 15 May 2016   Prob (F-statistic):             0.0198
Time:                        18:48:53   Log-Likelihood:                -712.13
No. Observations:                  66   AIC:                             1452.
Df Residuals:                      52   BIC:                             1483.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------------------
Intercept               

# Exercises

The following exercises are derived from Affective Events Theory (Weiss & Cropanzano, 1996).

1. Receiving a nasty email from a coworker leads to feelings of anger which reduce overall job satisfaction.

Does this describe mediation of moderation?

2. Access to free treats during break creates positive emotional responses for workers except for those who have high negative affect (i.e., who are perpetually grumpy).

Does this describe mediation or moderation?